In [1]:
import os
from os.path import join

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# 1.데이터 가져오기

In [2]:
data_dir = os.getenv('HOME')+'/aiffel/Project/hackathon1/data'

# 파일명
cats = join(data_dir, 'item_categories.csv')
items = join(data_dir, 'items.csv') 
shops = join(data_dir, 'shops.csv')
train = join(data_dir, 'sales_train.csv') 
test = join(data_dir, 'test.csv') 
sample_submission = join(data_dir, 'sample_submission.csv') 

# 데이터프레임
cats = pd.read_csv(cats, encoding='utf-8')
items = pd.read_csv(items, encoding='utf-8')
shops = pd.read_csv(shops, encoding='utf-8')
train = pd.read_csv(train, encoding='utf-8') # 훈련데이터셋
test = pd.read_csv(test, encoding='utf-8') # 테스트데이터셋
sample_submission = pd.read_csv(sample_submission, encoding='utf-8') #제출샘플 

# 2. 테이블별 데이터 정제

## 2-1. sales_train 데이터셋 정제 (train)

In [3]:
import os
from os.path import join

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# 1.데이터 가져오기

data_dir = os.getenv('HOME')+'/aiffel/Project/hackathon1/data'

# 파일명
cats = join(data_dir, 'item_categories.csv')
items = join(data_dir, 'items.csv') 
shops = join(data_dir, 'shops.csv')
train = join(data_dir, 'sales_train.csv') 
test = join(data_dir, 'test.csv') 
sample_submission = join(data_dir, 'sample_submission.csv') 

# 데이터프레임
cats = pd.read_csv(cats, encoding='utf-8')
items = pd.read_csv(items, encoding='utf-8')
shops = pd.read_csv(shops, encoding='utf-8')
train = pd.read_csv(train, encoding='utf-8') # 훈련데이터셋
test = pd.read_csv(test, encoding='utf-8') # 테스트데이터셋
sample_submission = pd.read_csv(sample_submission, encoding='utf-8') #제출샘플 

# 2. 테이블별 데이터 정제

## 2-1. sales_train 데이터셋 정제 (train)

## 2-2. shops 데이터셋 정제

In [4]:
shops.head() # 정제 전

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [5]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


- shop_id 재설정 ??

In [6]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

오타이거나(10, 11)..., 
하나의 상점이 폐업일 수도 있어서(0, 57) 또는 하나의 상점이 매출이 저조해서 통합한걸로 예상 (1, 58)
하지만 오타일 가능성이 크다.


- shop_name 을 공백을 기준으로 city와 category로 분리 

In [7]:
shops.loc[ shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"' # 도시 이름이 떨어져 있는거 붙여주기 
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
shops["category"] = shops.shop_name.str.split(" ").map( lambda x: x[1] )
shops.loc[shops.city == "!Якутск", "city"] = "Якутск" # 도시 이름에 ! 들어간거 정제

In [8]:
shops.category.unique()

array(['Орджоникидзе,', 'ТЦ', 'ТРК', 'ТРЦ', '(Плехановская,', 'Торговля',
       'ул.', 'ЧС', '"Распродажа"', 'МТРЦ', 'Магазин', 'ТК', 'склад'],
      dtype=object)

- 상점이 5개 이상인 것만 카테고리로 분류하고 나머지는 other 로 정제

분포 그래프 필요할거 같다. 왜 5를 기준으로 했는지

In [9]:
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5:
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )

In [10]:
shops.category.unique() 

array(['other', 'ТЦ', 'ТРК', 'ТРЦ', 'ТК'], dtype=object)

In [11]:
shops.head() # 정제 후

,shop_name,shop_id,city,category
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,other
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,ТЦ
2,"Адыгея ТЦ ""Мега""",2,Адыгея,ТЦ
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,ТРК
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,ТЦ


In [12]:
#shops

In [13]:
data_dir = os.getenv('HOME')+'/aiffel/hackathon/shops_map'  #'/Project/hackathon1/data'
shops_map_df  = join(data_dir, 'shops_map.csv')

shops_map_df = pd.read_csv(shops_map_df , encoding='utf-8')

In [14]:
#shops_map_df 

NaN값 때문에 지도에서 에러가 나서 일단 NaN값 드랍했습니다.

ValueError: Location values cannot contain NaNs.

In [15]:
shops_map_df2=shops_map_df.dropna().copy()

### shop들의 위치를 기반으로 분석 해보기

- 지도에 시각화를 시키기 위해 지도시각화 패키지인 folium을 설치합니다.

- 1차적으로 지도에 shop들을 찍어서 나타내 보겠습니다.

1. 설치하기


OpenMap : 일반 지도 형태

Stament Terrian : 도로선이 강조된 지도

Stametn Toner : 흑백지도

​

1) 지도 화면 불러 오기

: 가장 기본적으로 지도를 먼저 불러와야 합니다.

-  location : 필수,중심(위/경도 좌표)위치, list 형태  

-  zoom_start : 지도의 시작 화면 크기, 기본값10

-  tiles :  지도 Type(Basemap): 기본 open map
지도 종류는 아래 3번, 세부 파라미터는 4번 항목 참조

In [16]:
#pip install folium

#pip show folium

#conda install geopandas   #geopandas 설치 

In [17]:
import geopandas as gpd
from geopandas.tools import geocode


import folium  # folium은 호출합니다.
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Marker

from folium.plugins import HeatMap
from branca.colormap import linear



In [18]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [19]:
Russia_location = [64.0914, 101.6016]  # 지도의 중심 좌표,여기는 러시아 좌표입니다. 
m=folium.Map(location= Russia_location, zoom_start=3) 
# m변수 지정, 러시아, 화면 해상도는 3으로 지정함.
m     # m을 표시함. 


In [20]:
shops_map = folium.Map(location=[64.0914, 101.6016], tiles='Stamen Terrain', 
                        zoom_start=3)




for name, lat, lng in zip(shops_map_df2.shop_id,shops_map_df2.latitude,shops_map_df2.longitude):
    folium.CircleMarker([lat, lng],
                        radius=3,            
                        color='red',        
                        fill=True,
                        #fill_color='coral',   
                        #fill_opacity=0.7,   
                        popup=name
    ).add_to(shops_map)

# 지도를 html 파일로 저장하기
shops_map.save('./shops_map.html')

embed_map(shops_map, 'shops_map.html')

In [21]:
data_dir = os.getenv('HOME')+'/Downloads'  #'/Project/hackathon1/data'

train_map = join(data_dir,'train_map.csv')

train_map= pd.read_csv(train_map, encoding='utf-8')

In [22]:
train_map

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,shop_name,city,city_id,latitude,longitude
0,2013-01-02,0,59,22154,999.00,1.0,999.00,"Ярославль ТЦ ""Альтаир""",Ярославль,31.0,57.69779,39.75896
1,2013-01-03,0,25,2552,899.00,1.0,899.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
2,2013-01-05,0,25,2552,899.00,0.0,0.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
...,...,...,...,...,...,...,...,...,...,...,...,...
2935840,2015-10-10,33,25,7409,299.00,1.0,299.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
2935841,2015-10-09,33,25,7460,299.00,1.0,299.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
2935842,2015-10-14,33,25,7459,349.00,1.0,349.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908
2935843,2015-10-22,33,25,7440,299.00,1.0,299.00,"Москва ТРК ""Атриум""",Москва,14.0,55.75726,37.65908


In [26]:
rus_bnd_gdf = gpd.read_file('./github/Project1/data/geo.json')

DriverError: ./github/Project1/data/geo.json: No such file or directory

In [ ]:
# Regions mapping
rus_gdf = gpd.GeoDataFrame(train_map, geometry=gpd.points_from_xy(train_map.longitude,train_map.latitude))
rus_gdf.crs = {'init': 'epsg:4326'}

rus_shape = rus_gdf[['revenue', 'shop_id', 'city', 'geometry']]
rus_gdf = gpd.sjoin(rus_gdf, rus_shape, how="left", op='within')


# rus_gdf.loc[np.isnan(rus_gdf.shop_id)]
rus_gdf.head()

In [ ]:
m_3 = folium.Map(
    location=[64.0914, 101.6016],
    tiles='Stamen Toner',
    zoom_start=3
)

scale_min, scale_max = np.log(rus_gdf.revenue.min() + 1), np.log(rus_gdf.revenue.max() + 1)
colormap = linear.YlOrRd_09.scale(scale_min, scale_max)

def color_mapper(id):
    row = rus_gdf[rus_gdf.ID_1 == id].reset_index()
    if len(row) == 0:
        return scale_min
    return np.log(row.Act.iloc[0] + 1)

folium.GeoJson(
    rus_bnd_gdf,
    name='rusjson',
    style_function=lambda feature: {
        'fillColor': colormap(color_mapper(feature['properties']['shop_id'])),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m_3)

for i in range(len(rus_gdf)):
    folium.Circle(
        radius=20000,
        location=[rus_gdf.latitude[i], rus_gdf.longitude[i]],
        popup=rus_gdf.Region_en[i] + ' Active: {}'.format(int(rus_gdf.Act[i])),
        color='crimson',
        fill=True,
    ).add_to(m_3)

# folium.LatLngPopup().add_to(m_3)

# colormap.caption = 'Active cases color scale'
# colormap.add_to(m_3)

embed_map(m_3, 'm_3.html')